In [2]:
### IMPLEMENTATION OF BACKPROPAGAION IN TENSORFLOW 

# Reference : https://stackoverflow.com/questions/44210561/how-do-backpropagation-works-in-tensorflow
# How does backpropagation work in tensorflow?
import os
os.chdir('C:/Users/gupta/Desktop/DeepLearning/tutorials')

In [2]:
import tensorflow as tf

In [3]:
### constant data 
x =[[0.,0.],[1.,1.],[1.,0.],[0.,1.]]
y_ = [[0.],[0.],[1.],[1.]]

In [9]:
### induction
# 1x2 input -> 2x3 hidden sigmoid -> 3x1 sigmoid output

# Layer 0 = the x2 input
x0 = tf.constant(x, dtype = tf.float32)
y0 = tf.constant(y_, dtype = tf.float32)

In [10]:
# Layer 1 = the 2x3 hidden sigmoid
m1 = tf.Variable(tf.random_uniform([2,3],minval = 0.1, maxval = 0.9, dtype = tf.float32))
b1 = tf.Variable(tf.random_uniform([3], minval = 0.1, maxval = 0.9, dtype = tf.float32))
h1 = tf.sigmoid(tf.matmul(x0,m1) + b1)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
# Layer 2 = the 3x1 sigmoid output
m2 = tf.Variable(tf.random_uniform([3,1],minval = 0.1, maxval = 0.9, dtype = tf.float32))
b2 = tf.Variable(tf.random_uniform([1],minval = 0.1, maxval = 0.9, dtype = tf.float32))
y_out = tf.sigmoid(tf.matmul(h1, m2)+b2)

In [13]:
### loss 
# loss: sum of squares y_ - y_out
loss = tf.reduce_sum(tf.square(y0 - y_out))

In [14]:
# training step: gradient descent (1.0) to minimize loss
train = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

In [19]:
### training 
# run 500 times using all of X and Y
# print out loss and other interesting info
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(500):
        sess.run(train)
    
    results = sess.run([m1,b1,m2,b2,y_out,loss])
    labels = "m1,b1,m2,b2,y_out,loss".split(",")
    for label, result in zip(*(labels,results)):
        print("")
        print(label)
        print(result)
        
print("")


m1
[[2.7236252 5.965839  2.273778 ]
 [2.604435  6.0192814 2.4111016]]

b1
[-4.165486  -2.6090508 -3.6792357]

m2
[[-5.6930337]
 [ 8.323483 ]
 [-5.125822 ]]

b2
[-3.0668209]

y_out
[[0.06242108]
 [0.05551672]
 [0.94674855]
 [0.9467113 ]]

loss
0.012653899



In [3]:
os.chdir('C:/Users/gupta/Desktop/Case Study/kaggle/wheatSeeds')

In [4]:
### IMPLEMENTATION OF BACKPROPAGAION WITHOUT FANCY LIBRARIES
# reference: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [21]:
# Create a new neural network ready for training. 
# Initialize a network 
from random import random
from random import seed

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [22]:
seed(1)
net = initialize_network(2,1,2)
for layer in net:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [23]:
net

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'weights': [0.2550690257394217, 0.49543508709194095]},
  {'weights': [0.4494910647887381, 0.651592972722763]}]]

In [7]:
#net2 = initialize_network(4,3,2)
#for layer in net2:
#   print(layer)

In [8]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

In [9]:
activate([0.13436424411240122, 0.8474337369372327, 0.763774618976614],[1,0, None])

0.8981388630890152

In [10]:
from math import exp
# Transfer neuron activation 
def transfer(activation):
    return 1.0/(1.0 + exp(-activation))

In [24]:
# Forward propagation 
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer :
            activation = activate(neuron['weights'],inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [31]:
# testing forward propagation
net

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614],
   'output': 0.7105668883115941}],
 [{'weights': [0.2550690257394217, 0.49543508709194095],
   'output': 0.6629970129852887},
  {'weights': [0.4494910647887381, 0.651592972722763],
   'output': 0.7253160725279748}]]

In [26]:
row = [1,0, None]
output = forward_propagate(net, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [27]:
### Summary 
# In the above code, when you call forward_propagate following happens:
#  a) Starting with first hidden layer
#    i) neuron 1 : 
#        activation = sum of bias + ( weights * input)
#        transfer will give you sigmoid of activation
#    Same thing if there are more neurons in this layer, and then
#    new_inputs becomes the list of values that will be fed to next layer
# b) Starting with next hidden or output layer we apply the same process in recursion. 

In [28]:
def transfer_derivative(output):
    return output*(1.0-output)

In [29]:
# Back propagation
def back_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        # hidden layer
        if i!= len(network)-1: # i.e. for i not in output layer 
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        # output layer
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*transfer_derivative(neuron['output'])               

In [32]:
expected = [0,1]

In [34]:
back_propagate_error(net,expected)

In [36]:
for layer in net:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]
